__Tenant Churn Model to predict the tenant turnover with the past data__

- By: [Kathirvel Kumararaja]()

## Problem Statement

![title](../assets/churn.jpeg)  


Tenant Churn or Tenant turnover is one of the challenges if you are managing multiple properties with diversified tenants. Modeling tenant churn is a classical application of data science. To put in a simple language a churn model is a binary classification model that given a set of predictors, or input variables, outputs a classification. Our churn model will predict, if a tenant rents a unit, the probability that tenant will leave within a given time frame, for example 1 year. Armed with this knowledge a property manager can better understand how the tenant mix will change over time and which units are likely to become available soon. This will enable managers to proactively target the high churn candidates and provide incentives for them to extend their contract. In addition, the model can highlight which units likely will need to be filled in the future, so that the vacancy period of a unit can be reduced.

## Executive Summary

To address the challenge of finding the correct reddit post, our first strategy is to finalize a good reddit source for comparing the two popular sports Soccer and Cricekt. While most of us think Soccer as the most popular sports, data told us a different story. Cricket is played around 50 countries with 2.5 billion fans while football is played in over 200 countries with 3.5 billion fans. In other words per country there are 50 million cricket fans compared to 17.5 million football fans. It was a challenge to get a correct key word for the amount of data that we needed. We started with specific clubs of these sports and generalized to get enough data. 

Another challenge we were having was getting enough data (our target was close to 2000 records in each subreddit). I had to change the day window to attain this objective. Once I adjusted the day window to 15 and appropriate fequency I was able to collect enough data for training our models.

After extracting the blog post data next decision we had to make was what column or columns needs to be my X. We were inclined to use 'Title' and 'Self text' consdering the relevance of the contents. But after reviewing the shape of data and Null values we have decided to use only 'Title' as 'Self Text' was not adding much value in training my model.  

EDA helped me set up a preprocessing plan for our model. For preprocessing, I had created a fucntion that used regex, lemmatizatizer which removed punctuation and stopwords. For modeling, I used  TFIDF and Count vetorizers. They brought context of word choices into play, which will give us a better understanding of the group of words used in a reddit blog post about the sports we are analyzing.
 
Machines had no problem understanding the blog posts, after using vectorizers.

In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import time
import requests

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression

from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS as stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import confusion_matrix

import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from nltk.stem import PorterStemmer
import datetime as dt
import time

from sklearn.metrics import roc_auc_score

# Import CountVectorizer and TFIDFVectorizer from feature_extraction.text.
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import regex as re
import nltk
from nltk.stem import WordNetLemmatizer
import warnings
warnings.filterwarnings("ignore") 

#importing GaussianNB

from sklearn.naive_bayes import GaussianNB
from sklearn.base import TransformerMixin # this allows us to create a custom transformer

%matplotlib inline

/Users/kraja/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
#Read the data for resident info
df = pd.read_csv("../datasets/Tenant_Data_Edited_v1.csv",index_col=0)

In [3]:
df.head()

,Rent,Legal_Rent,Deposit,Lease _From,Lease_To,Move_In,Is_Past
Unit,,,,,,,
1-1A,"1,791.00","3,525.59",0.00,10/31/15,10/31/17,10/31/15,1
1-1A,"1,720.00","3,525.59","3,439.60",03/01/19,02/28/21,02/24/17,0
1-1C,"1,889.00","4,569.38",0.00,03/31/16,03/31/18,03/31/16,1
1-1C,"1,829.00","4,569.38","4,569.38",05/29/18,05/31/20,05/29/18,0
1-1E,0.00,"4,092.53",0.00,01/31/19,01/31/21,01/31/19,0


In [4]:
df.tail()

,Rent,Legal_Rent,Deposit,Lease _From,Lease_To,Move_In,Is_Past
Unit,,,,,,,
JAS-UNT1,0.00,NaN,0.00,02/28/15,02/28/17,02/28/15,1
JAS-UNT1,0.00,NaN,0.00,03/31/15,03/31/17,03/31/15,1
JAS-UNT1,0.00,NaN,0.00,03/31/15,03/31/17,03/31/15,1
JAS-UNT1,0.00,NaN,0.00,04/20/15,04/30/17,04/20/15,1
JAS-UNT1,0.00,NaN,0.00,05/30/15,05/31/17,05/30/15,1


In [5]:
df.shape

(8038, 7)